#### use the IMDB dataset that contains the text of 50,000 movie reviews

In [ ]:
import tensorflow as tf
from tensorflow import keras

import numpy as np
print(tf.__version__)

In [ ]:
imdb = keras.datasets.imdb

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000)

In [ ]:
# the dataset has been preprocessed by huffman coding, and all the words has been represented
# by a unique sequence of digits
print("Training entries: {}, labels: {}".format(len(train_data), len(train_labels)))

In [ ]:
print(train_data[0])

In [ ]:
# convert the integers back to words

# a dictionary mapping words to an integer index
word_index = imdb.get_word_index()

# the first indices are reserved
word_index = {k:(v+3) for k, v in word_index.items()}
word_index["<PAD>"] = 0
word_index["<START>"] = 1
word_index["<UNK>"] = 2 # unknown
word_index["<UNUSED>"] = 3

reserve_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

In [ ]:
decode_review(train_data[0])

In [ ]:
train_data = keras.preprocessing.sequence.pad_sequences(train_data, 
                                                       value=word_index["<PAD>"],
                                                       padding='post',
                                                       maxlen=256
                                                       )
test_data = keras.preprocessing.sequence.pad_sequences(test_data,
                                                      value=word_index["<PAD>"],
                                                      padding='post',
                                                      maxlen=256
                                                      )

In [ ]:
len(train_data[0]), len(train_data[1])

In [ ]:
print(train_data[0])

In [ ]:
# input shape is the vocabulary count used for the movie reviews (10,000 words)
vocab_size = 10000

model = keras.Sequential()
# The first layer is the Embedding layer.
# This layer takes the integer-encoded vocabulary and looks up the 
# embedding vector for each word-index.
model.add(keras.layers.Embedding(vocab_size, 16))
# GlobalAveragePooling1D layer returns a fixed-length output vector
# for each example by averaging over the sequence dimension.
# This allows the model to handle input of variable length.
model.add(keras.layers.GlobalAveragePooling1D())
# This fixed-length output vector is piped through a fully-connected
# Dense layer with 16 hidden units.
model.add(keras.layers.Dense(16, activation=tf.nn.relu))
# Densely connected with a single output node
model.add(keras.layers.Dense(1, activation=tf.nn.sigmoid))

model.summary()

In [ ]:
# loss function and optimizer
model.compile(optimizer = tf.train.AdamOptimizer(),
             loss = 'binary_crossentropy', 
             metrics = ['accuracy'])

In [ ]:
# create a validation set
x_val = train_data[:10000]
partial_x_train = train_data[10000:]

y_val = train_labels[:10000]
partial_y_train = train_labels[10000:]

In [ ]:
# train the model
history = model.fit(partial_x_train,
                   partial_y_train,
                   epochs = 40,
                   batch_size = 512,
                   validation_data = (x_val, y_val),
                   verbose = 1)

In [ ]:
# evaluate the model
results = model.evaluate(test_data, test_labels)

print(results)

In [ ]:
# create a graph of accuracy and loss over time
history_dict = history.history
history_dict.keys()

import matplotlib.pyplot as plt

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

# "bo" is for "blue dot"
plt.plot(epochs, loss, 'bo', label='Training loss')
# b is for "solid blue line"
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

In [ ]:
plt.clf()   # clear figure
acc_values = history_dict['acc']
val_acc_values = history_dict['val_acc']

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.show()